In [1]:
import os
import pandas as pd
import nltk
import numpy as np

In [2]:
from importlib import reload

In [3]:
dataDirectoryPath = 'resources/datasets-v5/tasks-2-3/train'

In [4]:
dataframesPath = 'resources'

In [5]:
from nltk import tokenize
from nltk.util import ngrams

In [6]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import collections
from nltk.stem.porter import *
from nltk import word_tokenize
import string
import pickle

In [7]:
from resources.readability import Readability

In [8]:
from resources.feature_functions import Functions

In [9]:
def whatsbeendon(filename):
    pids = []
    try:
        with open(filename) as data:
            pids = [line.strip().split(",")[0] for line in data]
        return set(pids)
    except:
        return set(pids)

def make_str(seq):
    strseq = [str(s) for s in seq]
    return strseq

In [10]:
Functions = Functions()

In [11]:
outfile = "features_binary_classification.csv"
outpath = "resources/"

In [12]:
done = whatsbeendon(outfile)

In [13]:
cat_dict, stem_dict, counts_dict = Functions.load_LIWC_dictionaries()
liwc_cats = [cat_dict[cat] for cat in cat_dict]
pos_tags = ["CC","CD","DT","EX","FW","IN","JJ","JJR","JJS","LS","MD","NN","NNS","NNP","NNPS","PDT","POS","PRP","PRP$","RB","RBR","RBS","RP","SYM","TO","UH","WP$","WRB","VB","VBD","VBG","VBN","VBP","VBZ","WDT","WP"]

In [16]:
if len(done) == 0:
    with open(os.path.join(outpath, outfile), "w") as out:
#         seq = ("pid, source, date, Happiness, HarmVirtue, HarmVice, FairnessVirtue, FairnessVice, IngroupVirtue, IngroupVice, AuthorityVirtue, AuthorityVice, PurityVirtue, PurityVice, MoralityGeneral, bias_count, assertives_count, factives_count, hedges_count, implicatives_count, report_verbs_count, positive_op_count, negative_op_count, wneg_count, wpos_count, wneu_count, sneg_count, spos_count, sneu_count, TTR,vad_neg,vad_neu,vad_pos,FKE,SMOG,stop,wordlen,WC,NB_pobj,NB_psubj,quotes,Exclaim,AllPunc,allcaps",",".join(pos_tags),",".join(liwc_cats))
        seq = ("ArticleId:SentenceNumber, Happiness, HarmVirtue, HarmVice, FairnessVirtue, FairnessVice, IngroupVirtue, IngroupVice, AuthorityVirtue, AuthorityVice, PurityVirtue, PurityVice, MoralityGeneral, bias_count, assertives_count, factives_count, hedges_count, implicatives_count, report_verbs_count, positive_op_count, negative_op_count, wneg_count, wpos_count, wneu_count, sneg_count, spos_count, sneu_count, TTR,vad_neg,vad_neu,vad_pos,FKE,SMOG,stop,wordlen,WC,quotes,Exclaim,AllPunc,allcaps",",".join(pos_tags),",".join(liwc_cats),"avg_word_sim,avg_sent_sim,x_y,x_2,y_2,Propaganda")
        out.write(",".join(seq)+"\n")

In [14]:
# datasetB = pd.read_excel(dataframesPath + "/dataframebinary.xlsx")
dataset = pd.read_csv(dataframesPath + "/dataframebinary_with_simscores2.csv")
# datasetB = datasetB[['Sentence', 'Propaganda']]

In [15]:
dataset.head()

,Unnamed: 0,ArticleId,SentenceNumber,Sentence,Propaganda,avg_word_sim,avg_sent_sim,x_y,x_2,y_2
0,0,111111112,5,Pamela Geller and Robert Spencer co-founded an...,yes,0.131630,0.771643,0.101572,0.017327,0.595432
1,3936,111111112,9,A government spokesman said individuals whose ...,no,0.411146,0.771643,0.317258,0.169041,0.595432
2,3937,111111112,13,'Right decision',no,0.480583,0.771643,0.370838,0.230960,0.595432
3,3938,111111112,17,They were due to attend a march planned by the...,no,0.924318,0.771643,0.713243,0.854363,0.595432
4,3939,111111112,19,"Keith Vaz, chairman of the Home Affairs Select...",no,2.628984,0.771643,2.028637,6.911558,0.595432


In [17]:
dataset['ArticleId:SentenceNumber'] = dataset['ArticleId'].astype(str) + ":" + dataset['SentenceNumber'].astype(str)

In [18]:
dataset = dataset[['ArticleId:SentenceNumber', 'Sentence', 'avg_word_sim', 'avg_sent_sim', 'x_y', 'x_2', 'y_2', 'Propaganda']]

In [19]:
dataset.head()

,ArticleId:SentenceNumber,Sentence,avg_word_sim,avg_sent_sim,x_y,x_2,y_2,Propaganda
0,111111112:5,Pamela Geller and Robert Spencer co-founded an...,0.131630,0.771643,0.101572,0.017327,0.595432,yes
1,111111112:9,A government spokesman said individuals whose ...,0.411146,0.771643,0.317258,0.169041,0.595432,no
2,111111112:13,'Right decision',0.480583,0.771643,0.370838,0.230960,0.595432,no
3,111111112:17,They were due to attend a march planned by the...,0.924318,0.771643,0.713243,0.854363,0.595432,no
4,111111112:19,"Keith Vaz, chairman of the Home Affairs Select...",2.628984,0.771643,2.028637,6.911558,0.595432,no


In [20]:
len(dataset)

7867

In [21]:
x = dataset['Sentence']
y = dataset['Propaganda']

In [22]:
len(x)

7867

In [23]:
x.head()

0    Pamela Geller and Robert Spencer co-founded an...
1    A government spokesman said individuals whose ...
2                                    'Right decision' 
3    They were due to attend a march planned by the...
4    Keith Vaz, chairman of the Home Affairs Select...
Name: Sentence, dtype: object

In [24]:
len(y)

7867

In [25]:
pos_features_path = "resources/temp/"

In [26]:
cat_dict, stem_dict, counts_dict = Functions.load_LIWC_dictionaries()
liwc_cats = [cat_dict[cat] for cat in cat_dict]

In [27]:
x[2238]

'Cardinal Burke to Michael Matt: "Amoris Not An Exercise of Magisterium" (Now what?) '

In [28]:
pd.options.mode.chained_assignment = None

In [29]:
for pid in range(len(x)):
    if dataset['ArticleId:SentenceNumber'][pid] in done:
        continue
    else:
        print ((pid+1), " - working on", dataset['ArticleId:SentenceNumber'][pid])
#     if(pid == 2239):
#         continue
    
    x[pid] = x[pid].strip()
#     datasetB_X[pid] = datasetB_X[pid].replace("\u200b", " ")
#     datasetB_X[pid] = datasetB_X[pid].replace("\ufeff", " ")
#     datasetB_X[pid] = datasetB_X[pid].replace("\u202c", " ")
#     datasetB_X[pid] = datasetB_X[pid].replace("\u0301", " ")
#     datasetB_X[pid] = datasetB_X[pid].replace("\u0301", " ")
#     if len(datasetB_X[pid]) == 0:
#         print("Empty string at index : ", pid)
#         datasetB_X = datasetB_X[pid].fillna()
#         continue
     
#     quotes, Exclaim, AllPunc, allcaps = Functions.stuff_LIWC_leftout(pid, x[pid])
    quotes, Exclaim, AllPunc, allcaps = Functions.stuff_LIWC_leftout(dataset['ArticleId:SentenceNumber'][pid], x[pid])
    lex_div = Functions.ttr(x[pid])
    counts_norm = Functions.POS_features("input", x[pid], pos_features_path)
    counts_norm = [str(c) for c in counts_norm]
    counts_norm_liwc, liwc_cats = Functions.LIWC(x[pid], cat_dict, stem_dict, counts_dict)
    counts_norm_liwc = [str(c) for c in counts_norm_liwc]
    vadneg, vadneu, vadpos = Functions.vadersent(x[pid])
    
    fke, SMOG = Functions.readability(x[pid])
    stop, wordlen, WC = Functions.wordlen_and_stop(x[pid])
#     print ("Functions.wordlen_and_stop(x[pid]) Done")
#     NB_pobj, NB_psubj = Functions.subjectivity(x[pid])
    bias_count, assertives_count, factives_count, hedges_count, implicatives_count, report_verbs_count, positive_op_count, negative_op_count, wneg_count, wpos_count, wneu_count, sneg_count, spos_count, sneu_count = Functions.bias_lexicon_feats(x[pid])
    HarmVirtue, HarmVice, FairnessVirtue, FairnessVice, IngroupVirtue, IngroupVice, AuthorityVirtue, AuthorityVice, PurityVirtue, PurityVice, MoralityGeneral = Functions.moral_foundation_feats(x[pid])
    happiness = Functions.happiness_index_feats(x[pid])
    
    with open(os.path.join(outpath, outfile), "a", encoding="utf-8") as out:
        print ("writing out")
#         seq = (pid, cat, date, happiness, HarmVirtue, HarmVice, FairnessVirtue, FairnessVice, IngroupVirtue, IngroupVice, AuthorityVirtue, AuthorityVice, PurityVirtue, PurityVice, MoralityGeneral, bias_count, assertives_count, factives_count, hedges_count, implicatives_count, report_verbs_count, positive_op_count, negative_op_count, wneg_count, wpos_count, wneu_count, sneg_count, spos_count, sneu_count, lex_div,vadneg,vadneu,vadpos,fke,SMOG,stop,wordlen,WC,NB_pobj,NB_psubj,quotes,Exclaim,AllPunc,allcaps, ",".join(counts_norm), ",".join(counts_norm_liwc))
        seq = (dataset['ArticleId:SentenceNumber'][pid], happiness, HarmVirtue, HarmVice, FairnessVirtue, FairnessVice, IngroupVirtue, IngroupVice, AuthorityVirtue, AuthorityVice, PurityVirtue, PurityVice, MoralityGeneral, bias_count, assertives_count, factives_count, hedges_count, implicatives_count, report_verbs_count, positive_op_count, negative_op_count, wneg_count, wpos_count, wneu_count, sneg_count, spos_count, sneu_count, lex_div,vadneg,vadneu,vadpos,fke,SMOG,stop,wordlen,WC,quotes,Exclaim,AllPunc,allcaps, ",".join(counts_norm), ",".join(counts_norm_liwc),dataset['avg_word_sim'][pid],dataset['avg_sent_sim'][pid],dataset['x_y'][pid],dataset['x_2'][pid],dataset['y_2'][pid],y[pid])
#         seq = (pid, lex_div,vadneg,vadneu,vadpos,stop,wordlen,WC,quotes,Exclaim,AllPunc,allcaps, ",".join(counts_norm), ",".join(counts_norm_liwc))
        seq = make_str(seq)
        feat_str = ",".join(seq)
        out.write(feat_str + "\n")
    print("Processing complete")

1  - working on 111111112:5
writing out
Processing complete
2  - working on 111111112:9
writing out
Processing complete
3  - working on 111111112:13
writing out
Processing complete
4  - working on 111111112:17
writing out
Processing complete
5  - working on 111111112:19
writing out
Processing complete
6  - working on 111111112:20
writing out
Processing complete
7  - working on 111111112:23
writing out
Processing complete
8  - working on 111111112:28
writing out
Processing complete
9  - working on 111111112:30
writing out
Processing complete
10  - working on 111111112:31
writing out
Processing complete
11  - working on 111111112:35
writing out
Processing complete
12  - working on 111111112:36
writing out
Processing complete
13  - working on 111111112:38
writing out
Processing complete
14  - working on 111111112:39
writing out
Processing complete
15  - working on 111111112:40
writing out
Processing complete
16  - working on 111111112:42
writing out
Processing complete
17  - working on 11

writing out
Processing complete
134  - working on 111111124:42
writing out
Processing complete
135  - working on 111111124:5
writing out
Processing complete
136  - working on 111111124:3
writing out
Processing complete
137  - working on 111111124:1
writing out
Processing complete
138  - working on 111111124:32
writing out
Processing complete
139  - working on 111111124:8
writing out
Processing complete
140  - working on 111111124:10
writing out
Processing complete
141  - working on 111111124:12
writing out
Processing complete
142  - working on 111111124:13
writing out
Processing complete
143  - working on 111111124:15
writing out
Processing complete
144  - working on 111111124:17
writing out
Processing complete
145  - working on 111111124:21
writing out
Processing complete
146  - working on 111111124:18
writing out
Processing complete
147  - working on 111111124:19
writing out
Processing complete
148  - working on 111111131:9
writing out
Processing complete
149  - working on 111111131:

writing out
Processing complete
267  - working on 111111132:6
writing out
Processing complete
268  - working on 111111132:7
writing out
Processing complete
269  - working on 111111132:8
writing out
Processing complete
270  - working on 111111132:10
writing out
Processing complete
271  - working on 111111132:12
writing out
Processing complete
272  - working on 111111132:13
writing out
Processing complete
273  - working on 111111132:15
writing out
Processing complete
274  - working on 111111132:62
writing out
Processing complete
275  - working on 111111132:46
writing out
Processing complete
276  - working on 111111132:64
writing out
Processing complete
277  - working on 111111132:112
writing out
Processing complete
278  - working on 111111132:114
writing out
Processing complete
279  - working on 111111132:119
writing out
Processing complete
280  - working on 111111132:120
writing out
Processing complete
281  - working on 111111132:121
writing out
Processing complete
282  - working on 111

writing out
Processing complete
398  - working on 694327499:78
writing out
Processing complete
399  - working on 694327499:79
writing out
Processing complete
400  - working on 694327499:84
writing out
Processing complete
401  - working on 694327499:82
writing out
Processing complete
402  - working on 694327499:59
writing out
Processing complete
403  - working on 694327499:58
writing out
Processing complete
404  - working on 694327499:53
writing out
Processing complete
405  - working on 694327499:46
writing out
Processing complete
406  - working on 694327499:33
writing out
Processing complete
407  - working on 694327499:28
writing out
Processing complete
408  - working on 694327499:80
writing out
Processing complete
409  - working on 694327499:26
writing out
Processing complete
410  - working on 694327499:5
writing out
Processing complete
411  - working on 694327499:72
writing out
Processing complete
412  - working on 694327499:56
writing out
Processing complete
413  - working on 694327

writing out
Processing complete
529  - working on 696246189:29
writing out
Processing complete
530  - working on 696246189:25
writing out
Processing complete
531  - working on 696246189:24
writing out
Processing complete
532  - working on 696246189:23
writing out
Processing complete
533  - working on 696246189:22
writing out
Processing complete
534  - working on 696246189:21
writing out
Processing complete
535  - working on 696246189:20
writing out
Processing complete
536  - working on 696246189:19
writing out
Processing complete
537  - working on 696246189:18
writing out
Processing complete
538  - working on 696246189:26
writing out
Processing complete
539  - working on 696246189:16
writing out
Processing complete
540  - working on 696246189:17
writing out
Processing complete
541  - working on 696246189:3
writing out
Processing complete
542  - working on 696246189:4
writing out
Processing complete
543  - working on 696246189:6
writing out
Processing complete
544  - working on 69624618

...

Processing complete
7263  - working on 790483991:57
writing out
Processing complete
7264  - working on 790483991:56
writing out
Processing complete
7265  - working on 790483991:50
writing out
Processing complete
7266  - working on 790483991:69
writing out
Processing complete
7267  - working on 790483991:49
writing out
Processing complete
7268  - working on 790483991:43
writing out
Processing complete
7269  - working on 790483991:4
writing out
Processing complete
7270  - working on 790483991:48
writing out
Processing complete
7271  - working on 790483991:8
writing out
Processing complete
7272  - working on 790483991:10
writing out
Processing complete
7273  - working on 790483991:11
writing out
Processing complete
7274  - working on 790483991:6
writing out
Processing complete
7275  - working on 790483991:14
writing out
Processing complete
7276  - working on 790483991:15
writing out
Processing complete
7277  - working on 790483991:36
writing out
Processing complete
7278  - working on 7904

writing out
Processing complete
7394  - working on 795379208:24
writing out
Processing complete
7395  - working on 795379208:14
writing out
Processing complete
7396  - working on 795379208:10
writing out
Processing complete
7397  - working on 795379208:11
writing out
Processing complete
7398  - working on 795689901:7
writing out
Processing complete
7399  - working on 795689901:4
writing out
Processing complete
7400  - working on 795691173:18
writing out
Processing complete
7401  - working on 795691173:7
writing out
Processing complete
7402  - working on 795693029:15
writing out
Processing complete
7403  - working on 795693029:10
writing out
Processing complete
7404  - working on 795693029:5
writing out
Processing complete
7405  - working on 795693029:14
writing out
Processing complete
7406  - working on 795703371:11
writing out
Processing complete
7407  - working on 795703371:39
writing out
Processing complete
7408  - working on 795703371:36
writing out
Processing complete
7409  - work

Processing complete
7523  - working on 999001619:39
writing out
Processing complete
7524  - working on 999001619:41
writing out
Processing complete
7525  - working on 999001619:45
writing out
Processing complete
7526  - working on 999001619:43
writing out
Processing complete
7527  - working on 999001619:80
writing out
Processing complete
7528  - working on 999001619:81
writing out
Processing complete
7529  - working on 999001619:82
writing out
Processing complete
7530  - working on 999001619:83
writing out
Processing complete
7531  - working on 999001619:85
writing out
Processing complete
7532  - working on 999001619:86
writing out
Processing complete
7533  - working on 999001619:100
writing out
Processing complete
7534  - working on 999001619:99
writing out
Processing complete
7535  - working on 999001619:98
writing out
Processing complete
7536  - working on 999001619:97
writing out
Processing complete
7537  - working on 999001619:24
writing out
Processing complete
7538  - working on 

writing out
Processing complete
7652  - working on 7383612089:15
writing out
Processing complete
7653  - working on 7383612089:16
writing out
Processing complete
7654  - working on 7383612089:20
writing out
Processing complete
7655  - working on 7383612089:22
writing out
Processing complete
7656  - working on 7383612089:23
writing out
Processing complete
7657  - working on 7383612089:27
writing out
Processing complete
7658  - working on 7383612089:18
writing out
Processing complete
7659  - working on 7383612089:29
writing out
Processing complete
7660  - working on 7383612089:31
writing out
Processing complete
7661  - working on 7383612089:25
writing out
Processing complete
7662  - working on 7383612089:24
writing out
Processing complete
7663  - working on 7383612089:26
writing out
Processing complete
7664  - working on 7383612089:21
writing out
Processing complete
7665  - working on 7383612089:19
writing out
Processing complete
7666  - working on 7383612089:3
writing out
Processing com

Processing complete
7779  - working on 7646642839:26
writing out
Processing complete
7780  - working on 7646642839:27
writing out
Processing complete
7781  - working on 7646642839:65
writing out
Processing complete
7782  - working on 7646642839:25
writing out
Processing complete
7783  - working on 7646642839:22
writing out
Processing complete
7784  - working on 7646642839:20
writing out
Processing complete
7785  - working on 7646642839:17
writing out
Processing complete
7786  - working on 7646642839:16
writing out
Processing complete
7787  - working on 7646642839:15
writing out
Processing complete
7788  - working on 7646642839:14
writing out
Processing complete
7789  - working on 7646642839:12
writing out
Processing complete
7790  - working on 7646642839:10
writing out
Processing complete
7791  - working on 7646642839:9
writing out
Processing complete
7792  - working on 7646642839:5
writing out
Processing complete
7793  - working on 7646642839:30
writing out
Processing complete
7794  -